# Import Libraries

In [5]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model

import pickle, os


In [1]:
NUM_HEROES = 123
NUM_FEATURES = NUM_HEROES * 2

class NNPredictor:
    def __init__(self, model_root = 'neural_network'):
        model_path = os.path.join(model_root, 'model.pkl')
        with open(model_path, 'r') as input_file:
            self.model = pickle.load(input_file)

    def transform(self, my_team, their_team):
        X = np.zeros(NUM_FEATURES, dtype=np.int8)
        for hero_id in my_team:
            X[hero_id - 1] = 1
        for hero_id in their_team:
            X[hero_id - 1 + NUM_HEROES] = 1
        return X

    def recommend(self, my_team, their_team, hero_candidates):
        team_possibilities = [(candidate, my_team + [candidate]) for candidate in hero_candidates]

        prob_candidate_pairs = []
        for candidate, team in team_possibilities:
            query = self.transform(team, their_team)
            prob = self.score(query)
            prob_candidate_pairs.append((prob, candidate))
        prob_candidate_pairs = sorted(prob_candidate_pairs, reverse = True)[0:5 - len(my_team)]
        return prob_candidate_pairs

    def score(self,query):
        radiant_query = query
        rad_prob = self.model.predict_proba(radiant_query)[0][1]
        return rad_prob

    def predict(self, dream_team, their_team):
        '''Returns the probability of the dream_team winning against their_team.'''
        dream_team_query = self.transform(dream_team, their_team)
        return self.score(dream_team_query)
            #return self.model.predict_proba(dream_team_query)[0][1]